In [64]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

import numpy as np
import pandas as pd
import random

import networkx as nx
import jReversion as jR
from LDOI import BooleanDOI_processing as BDOIp

In [65]:
BooleanRuleFileName = '../models/blt.txt'
NetworkName = 'blt'
Prefix, Suffix = 'n', 'n'
TempI = jR.initialize(BooleanRuleFileName, Prefix, Suffix)
Mapping = TempI['mapping']
InverseMapping = TempI['inverse_mapping']
GRead = TempI['Gread']
ReadNodes = TempI['read_nodes']



In [141]:
input_nodes = [node for node in GRead.nodes if GRead.in_degree(node) == 0]
output_nodes = [node for node in GRead.nodes if GRead.out_degree(node) == 0]

num_inputs = len(input_nodes)
num_input_conditions = 2 ** num_inputs
InputConditions = np.ndarray((num_inputs, 2), dtype=object)
for idx, input_node in enumerate(input_nodes):
    InputConditions[idx, 0] = '~' + ReadNodes[input_node]
    InputConditions[idx, 1] = ReadNodes[input_node]


OutputNodes = []
OutputNodes.extend([ReadNodes[idx] for idx in output_nodes])
OutputNodes.extend(['~'+ReadNodes[idx] for idx in output_nodes])
InputNodes = InputConditions.reshape(num_inputs * 2, ).tolist()

In [138]:
# InputConditions = np.ndarray((1,2), dtype=object)
# InputConditions[0,0] = '~Estrogen'
# InputConditions[0,1] = 'Estrogen'
# InputNodes = InputConditions.reshape(1 * 2, ).tolist()

In [155]:
GExpanded = BDOIp.Get_expanded_network(GRead, prefix=Prefix, suffix=Suffix)

In [184]:
OutputNodes = ['KRT18', '~KRT18', 'C6orf97', '~C6orf97', 'CA12', '~CA12', 'ESR1', '~ESR1']

In [185]:
# integrated_data = pd.read_csv('../data/' + network_name + '_table_for_original_network.tsv', sep='\t').sort_values(by='node').set_index('node')
# table_original_network = table_original_network.sort_values(by='node')
# ffl_test = pd.read_csv('../data/' + network_name + '_ffl_test.tsv', sep='\t').sort_values(by='node').set_index('node')
# canalizing_test = pd.read_csv('../data/' + network_name + '_canalizing_check_for_original_network.tsv', sep='\t').sort_values(by='node').set_index('node')

TempGIOW = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                            constant_nodes=[])
LDOIs = TempGIOW['LDOIs']
GeneLDOIs = TempGIOW['gene_LDOIs']
Conflicts = TempGIOW['conflicts']
GeneConflicts = TempGIOW['gene_conflicts']
IORelation = TempGIOW['io_relation']
GRemained = TempGIOW['G_remained']

R0 = jR.identifying_r0_mutations_ldoi(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions, IORelation)
# DM = jR.identifying_disconnecting_mutations(GExpanded, InputNodes, OutputNodes, Mapping, InverseMapping)
R1 = jR.identifying_r1_mutations_ldoi(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions, IORelation)
#     IIDC = jR.identifying_input_independent_canalizing_mutations(GExpanded, OutputNodes, Mapping, InverseMapping)
#     UN = jR.identifying_input_unreachable_nodes(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions)
RN = jR.identifying_rn_mutations(GExpanded, Mapping, InverseMapping, InputNodes, OutputNodes, InputConditions,
                                 IORelation,
                                 R0['ineffective_mutations'], R1['r1_mutations'])

NodeList = set(ReadNodes.values())
NodeList.difference_update(InputNodes)
NodeList.difference_update(OutputNodes)
C0, C1, C2, C3 = [], [], [], []

node_classificatin_dict = {}

for NODE in NodeList:
    negNODE = '~' + NODE
    if R0['ineffective'][NODE] and R0['ineffective'][negNODE]:
#             nodeClass = 'C0'
        node_classificatin_dict[NODE] = 'C0'
    elif NODE in RN['rn_mutations']:
        if RN['rn_mutations'][NODE] == 'R1':
#                 nodeClass = 'C1'
            node_classificatin_dict[NODE] = 'C1'
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
        else:
#                 nodeClass = 'C2'
            node_classificatin_dict[NODE] = 'C2'
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
    elif negNODE in RN['rn_mutations']:
        if RN['rn_mutations'][negNODE] == 'R1':
#                 nodeClass = 'C1'
            node_classificatin_dict[NODE] = 'C1'
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
    else:
#             nodeClass = 'C3'
        node_classificatin_dict[NODE] = 'C3'
#             canalizing = IIDC['iid_canalizing'][NODE]
#             unreachable = UN['input_unreachable'][NODE]

In [186]:
print(node_classificatin_dict['HDAC2'])
print(node_classificatin_dict['BCL11A'])

C1
C1


In [190]:
node_classificatin_dict

{'c_Src': 'C0',
 'FOXA1': 'C0',
 'HDAC2': 'C1',
 'C4orf7': 'C0',
 'ANXA9': 'C0',
 'GATA3': 'C0',
 'CDCA7': 'C0',
 'BCL11A': 'C1',
 'KRAS': 'C0',
 'EGFR': 'C0',
 'MSLN': 'C0',
 'ZEB1': 'C0',
 'CDH1': 'C0',
 'PI3K': 'C0',
 'STAT5': 'C0',
 'SOX11': 'C0',
 'ERK1_2': 'C0',
 'CCNE1': 'C1',
 'SNAI1': 'C0',
 'FOXC1': 'C1',
 'ER': 'C1',
 'PTEN': 'C0',
 'AKT': 'C0',
 'SLUG': 'C0'}

In [189]:
IORelation

array([[list([]), list([]), list([]), list([])]], dtype=object)

In [191]:
GLabel = nx.DiGraph()
GLabel.add_edges_from(GRead.edges())
GLabel = nx.relabel_nodes(GLabel, ReadNodes)

nx.set_node_attributes(G=GLabel, name='class', values=node_classificatin_dict)
path = '../networks/' + NetworkName + '_topology.gml'
with open(path, 'wb') as f:
    nx.write_gml(G=GLabel, path=f)

In [70]:
tt = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, ['ESR1', '~ESR1', 'ERK1_2', '~ERK1_2'],
                                            constant_nodes=[])
print(tt['io_relation'])

[[list([]) list([]) list([]) list([])]]


In [71]:
# tt = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
#                                             constant_nodes=['~BCL11A', '~HDAC2'])

In [128]:

TempGIOW = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                            constant_nodes=[])
LDOIs = TempGIOW['LDOIs']
GeneLDOIs = TempGIOW['gene_LDOIs']
Conflicts = TempGIOW['conflicts']
GeneConflicts = TempGIOW['gene_conflicts']
IORelation = TempGIOW['io_relation']
GRemained = TempGIOW['G_remained']

In [129]:
R0 = jR.identifying_r0_mutations_ldoi(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions, IORelation)
# DM = jR.identifying_disconnecting_mutations(GExpanded, InputNodes, OutputNodes, Mapping, InverseMapping)
R1 = jR.identifying_r1_mutations_ldoi(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions, IORelation)
#     IIDC = jR.identifying_input_independent_canalizing_mutations(GExpanded, OutputNodes, Mapping, InverseMapping)
#     UN = jR.identifying_input_unreachable_nodes(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions)
RN = jR.identifying_rn_mutations(GExpanded, Mapping, InverseMapping, InputNodes, OutputNodes, InputConditions,
                                 IORelation,
                                 R0['ineffective_mutations'], R1['r1_mutations'])

NodeList = set(ReadNodes.values())
NodeList.difference_update(InputNodes)
NodeList.difference_update(OutputNodes)
C0, C1, C2, C3 = [], [], [], []

node_classificatin_dict = {}

for NODE in NodeList:
    negNODE = '~' + NODE
    if R0['ineffective'][NODE] and R0['ineffective'][negNODE]:
#             nodeClass = 'C0'
        node_classificatin_dict[NODE] = 'C0'
    elif NODE in RN['rn_mutations']:
        if RN['rn_mutations'][NODE] == 'R1':
#                 nodeClass = 'C1'
            node_classificatin_dict[NODE] = 'C1'

        else:
#                 nodeClass = 'C2'
            node_classificatin_dict[NODE] = 'C2'
    elif negNODE in RN['rn_mutations']:
        if RN['rn_mutations'][negNODE] == 'R1':
#                 nodeClass = 'C1'
            node_classificatin_dict[NODE] = 'C1'
    else:
#             nodeClass = 'C3'
        node_classificatin_dict[NODE] = 'C3'

In [130]:
IORelation

array([[list([]), list([]), list([]), list([])]], dtype=object)

In [131]:
node_classificatin_dict

{'KRT18': 'C1',
 'c_Src': 'C0',
 'FOXA1': 'C0',
 'HDAC2': 'C2',
 'C4orf7': 'C1',
 'ANXA9': 'C0',
 'GATA3': 'C0',
 'CA12': 'C0',
 'CDCA7': 'C0',
 'BCL11A': 'C2',
 'KRAS': 'C3',
 'EGFR': 'C3',
 'MSLN': 'C1',
 'ZEB1': 'C0',
 'CDH1': 'C1',
 'PI3K': 'C0',
 'STAT5': 'C1',
 'SOX11': 'C1',
 'CCNE1': 'C2',
 'C6orf97': 'C0',
 'SNAI1': 'C0',
 'FOXC1': 'C1',
 'ER': 'C1',
 'PTEN': 'C0',
 'AKT': 'C1',
 'SLUG': 'C0'}

In [132]:
tt = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                            constant_nodes=['~BCL11A', '~KRT18'])

In [133]:
tt['io_relation']

array([[list([]), list([]), list([]), list([])]], dtype=object)

In [134]:
ttt = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                            constant_nodes=['HDAC2', 'C4orf7'])
print(ttt['io_relation'])

[[list([]) list([]) list([]) list([])]]


In [136]:
t = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                            constant_nodes=['~BCL11A', '~HDAC2'])
print(t['io_relation'])

[[list([]) list([]) list(['~ERK1_2']) list([])]]
